In [1]:
import nltk
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics 
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import permutation_test_score
import re
import random
from sklearn.linear_model import LogisticRegression

/Users/fatma/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /Users/fatma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " ", string)
    string = re.sub(r"\'ve", " ", string)
    string = re.sub(r"n\'t", " ", string)
    string = re.sub(r"n\'s", " ", string)
    string = re.sub(r"\'re", " ", string)
    string = re.sub(r"\'d", " ", string)
    string = re.sub(r"\'ll", " ", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"!", " ", string)
    string = re.sub(r"\(", " ", string)
    string = re.sub(r"\)", " ", string)
    string = re.sub(r"\?", " ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"\//?", " ", string)
    string = re.sub(r"\d+", " ", string)
    string = re.sub(r"\$", " ", string)
    string = re.sub(r"\#", " ", string)
    return string.strip().lower()

In [7]:
def remove_stop(str):
    stop = set(stopwords.words('english'))
    lst = str.split()
    lst = [i for i in lst if i not in stop]
    return ' '.join(lst)

def stem (str):
    lst = str.split()
    lst = [st.stem(x) for x in lst]
    return ' '.join(lst)

def remove_unwanted_words(str):
    unwanted_words = ["httpaddress", "usrid", "D", "dd", "rt", "amp", "am", "pm", '``',
                      "''", "", "//", "\\", "\\'s", "\\?", "\?","http","httpaddresshttpaddresst", "cohttpaddressek",
                      "taksim","gezi", "park", "direngeziparki", "occupygezi", "istanbul", "turkish","turkey",
                      "protest","direngezipark","direnankara","geziparki", "protesters", "protests", "sat", "sun", "mon,",
                     "tue", "wed", "thu", "fri", "jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
    lst = str.split(" ")
    lst = [i for i in lst if i not in unwanted_words]
    return ' '.join(lst)

def toLower(str):
    lst = str.split()
    lst = [i.lower() for i in lst]
    return ' '.join(lst)

def word_len (str):
    lst = str.split()
    lst = [i for i in lst if len(i)>1 and len(i) <7]
    return ' '.join(lst)

def sent_len (str):
    lst = str.split()
    if len(lst)>=3:
        return ' '.join(lst)

In [57]:
def clean_data(data):
    """
    Loads MR polarity african_data from files, splits the african_data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load african_data from files
    positive_examples = data
    positive_examples = positive_examples.strip() 
    positive_examples = remove_stop(positive_examples) 
    #print(positive_examples[0])
    positive_examples = toLower(positive_examples) 
    #rint(positive_examples[0])
    positive_examples = clean_str(positive_examples) 
    #rint(positive_examples[0])
    #positive_examples = [stem(item) for item in positive_examples]
    #rint(positive_examples[0])
    positive_examples = remove_unwanted_words(positive_examples)
    positive_examples = word_len(positive_examples) 
    positive_examples = sent_len(positive_examples) 
    
    return positive_examples
    

In [74]:
df_cf_judgments = pd.read_csv('Data/CF_confidence_judgments_finalized.csv')

In [75]:
df_cf_judgments.columns

Index(['_unit_id', 'text_x', 'text_y', 'proccd_text_x', 'protest_x',
       'protest_y', 'protest_confidence', 'violence_x', 'violence_y',
       'violence_confidence', 'created_at_x', '_golden_x'],
      dtype='object')

In [76]:
df_cf_judgments = df_cf_judgments[df_cf_judgments['_golden_x']==False]

In [77]:
df_cf_ready_ds = df_cf_judgments[['_unit_id','text_x','proccd_text_x','violence_y','created_at_x']]

In [78]:
df_cf_ready_ds.rename(columns={'text_x':'text', 'proccd_text_x':'proccd_text',
                                                'violence_y':'violence','created_at_x':'created_at'}, inplace = True)

/Users/fatma/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [79]:
len(df_cf_ready_ds)

1315

In [80]:
df_cf_ready_ds['violence'] = df_cf_ready_ds['violence'].fillna(0)

/Users/fatma/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [82]:
df_cf_ready_ds['clean_text'] = df_cf_ready_ds_pos['text'].apply(lambda x:clean_data(x) )

/Users/fatma/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [87]:
df_cf_ready_ds['clean_text'] = df_cf_ready_ds['clean_text'].dropna()

/Users/fatma/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [88]:
df_cf_ready_ds

,_unit_id,text,proccd_text,violence,created_at,clean_text
3,1657977942,RT @MarietjeSchaake: @MarkABentley: Erdogan sa...,rt usrId usrId erdogan says vandals entering m...,1.0,Sun Jun 09 14:53:50 +0000 2013,says beer women
11,1657977950,"Turkey police crush protests in Ankara, Istanb...",turkey police crush protests in ankara istanbu...,1.0,Mon Jun 10 03:49:59 +0000 2013,police crush ankara photos video
14,1657977954,RT @skyyfitsheaven: Police detaining protestor...,rt usrId police detaining protestors in #diren...,1.0,Wed Jun 12 23:29:49 +0000 2013,police asked open apt doors hid
16,1657977956,CBS Riot police rush to crush renewed Turkey p...,cbs riot police rush to crush renewed turkey p...,1.0,Sun Jun 16 11:41:34 +0000 2013,cbs riot police rush crush co news
25,1657977970,Protesters blocking bulldozers tear gassed in ...,protesters blocking bulldozers tear gassed in ...,1.0,Fri May 31 14:35:57 +0000 2013,tear gassed co riv
28,1657977974,"""@paulocoelho: Shame on us: tear gas used in T...",paulocoelho shame on us tear gas used in turke...,1.0,Tue Jun 04 22:19:42 +0000 2013,shame us tear gas used made brasil co gee lnen
36,1657977983,"Protests in Istanbul, Turkey: Photographs of T...",protests in istanbul turkey photographs of tur...,1.0,Fri May 31 23:17:31 +0000 2013,police using tear gas water cannon people co z...
41,1657977988,RT @Timcast: In #Istanbul reporting for @Vice....,rt usrId in #istanbul reporting for httpAddres...,1.0,Mon Jun 10 02:06:18 +0000 2013,in vice police launch live happen
43,1657977991,RT @zeynep: I think this is the last pic I man...,rt usrId i think this is the last pic i manage...,1.0,Tue Jun 11 18:30:17 +0000 2013,zeynep think last pic take see gas cloud middl...
49,1657977998,RT @Anon_Online: #Istanbul #Protestors holding...,rt usrId #istanbul #protestors holding their g...,1.0,Sun Jun 16 19:34:30 +0000 2013,anon online ground police no


In [89]:
test_predict_auc_all =[]
for i in range(1,10):
    df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_cf_ready_ds.clean_text, 
                                                    df_cf_ready_ds.violence, 
                                                    test_size=0.5)
    df_train_df = pd.concat([df_X_train,df_y_train],axis=1)
    df_test_df = pd.concat([df_X_test,df_y_test],axis = 1)
    print('training set size', len(df_train_df))
    print('training pos', len(df_train_df[df_train_df["violence"]==1]))
    print('training neg', len(df_train_df[df_train_df["violence"]==0]))
    print('test set size', len(df_test_df))
    print('test pos', len(df_test_df[df_test_df["violence"]==1]))
    print('test neg', len(df_test_df[df_test_df["violence"]==0]))
    
    df_pipeline_violence_Svc = Pipeline([
    ('vectorizer',  CountVectorizer(ngram_range=(1, 2),stop_words='english',  min_df=3)),
    ('classifier',  SVC(kernel='linear', probability=True)) ])

    
    df_scores = cross_val_score(df_pipeline_violence_Svc, df_train_df.clean_text, df_train_df.violence, cv=10)
    print('scores',df_scores)
    print('scores mean',df_scores.mean())
    print('scores std',df_scores.std() * 2)
    
    df_cv_predict = cross_val_predict(df_pipeline_violence_Svc, df_train_df.clean_text, df_train_df.violence, cv=10)
    df_cv_predict_scores = metrics.roc_auc_score(df_train_df.violence, df_cv_predict)
    print('cv auc', df_cv_predict_scores)
    
    df_pipeline_violence_Svc_model = df_pipeline_violence_Svc.fit(df_train_df.clean_text, df_train_df.violence)
    df_prediction = df_pipeline_violence_Svc_model.predict(df_test_df.clean_text)
    df_predict_prob = df_pipeline_violence_Svc_model.predict_proba(df_test_df.clean_text)
    test_predict_auc = metrics.roc_auc_score(df_test_df.violence,df_prediction)
    print('test set auc', test_predict_auc)
    test_predict_auc_all.append(test_predict_auc)
    print(metrics.confusion_matrix(df_test_df.violence,df_prediction))

training set size 102
training pos 102
training neg 0
test set size 103
test pos 103
test neg 0


ValueError: The number of classes has to be greater than one; got 1